In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
PATH = './input/'
df = pd.read_csv(PATH + "webtoon_total_final.csv")

In [3]:
df = df[['title','score', 'genre']]
df.genre = df.genre.str.strip('['']')

In [4]:
tmp = pd.get_dummies(df.genre)

In [5]:
def col_change(df):
    for col in df.columns:
        if ',' in col:
            col_sep = col.split(', ')
            df[col_sep[0]] = df[col_sep[0]] + df[col]
            df[col_sep[1]] = df[col_sep[1]] + df[col]
            df.drop(columns=[f'{col}'], inplace=True)
    return df

In [6]:
col_change(tmp)
tmp

,'드라마','로맨스','무협','스릴러','액션','일상','코믹','판타지','학원'
0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,1,0
2,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
3225,0,1,0,0,0,0,0,0,0
3226,0,0,0,1,0,0,0,0,0
3227,0,1,0,0,0,0,0,0,0
3228,0,0,0,0,0,0,0,1,1


In [7]:
genre_df = col_change(pd.get_dummies(df.genre))
df = pd.concat([df.title, df.score, genre_df], axis=1)
df

,title,score,'드라마','로맨스','무협','스릴러','액션','일상','코믹','판타지','학원'
0,참교육,9.956667,0,0,0,0,1,0,0,0,0
1,신의 탑,6.216667,0,0,0,0,0,0,0,1,0
2,뷰티풀 군바리,9.780000,1,0,0,0,0,0,0,0,0
3,윈드브레이커,9.783333,0,0,0,0,0,0,0,0,1
4,퀘스트지상주의,9.876667,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
3225,진심이 닿다,9.776667,0,1,0,0,0,0,0,0,0
3226,드림사이드,9.836667,0,0,0,1,0,0,0,0,0
3227,김 비서가 왜 그럴까,9.933333,0,1,0,0,0,0,0,0,0
3228,더스크 하울러,9.933333,0,0,0,0,0,0,0,1,1


In [8]:
df.columns = df.columns.str.strip('\'')
df

,title,score,드라마,로맨스,무협,스릴러,액션,일상,코믹,판타지,학원
0,참교육,9.956667,0,0,0,0,1,0,0,0,0
1,신의 탑,6.216667,0,0,0,0,0,0,0,1,0
2,뷰티풀 군바리,9.780000,1,0,0,0,0,0,0,0,0
3,윈드브레이커,9.783333,0,0,0,0,0,0,0,0,1
4,퀘스트지상주의,9.876667,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
3225,진심이 닿다,9.776667,0,1,0,0,0,0,0,0,0
3226,드림사이드,9.836667,0,0,0,1,0,0,0,0,0
3227,김 비서가 왜 그럴까,9.933333,0,1,0,0,0,0,0,0,0
3228,더스크 하울러,9.933333,0,0,0,0,0,0,0,1,1


In [9]:
def one_genre(title):
    g_row = df[df.title == title]
    genres = g_row[df.columns[2:]]
    return genres.values

def genres(title_list):
    genre_list = [0]*9
    for title in title_list:
        genre_list = genre_list + one_genre(title)
    return genre_list


## 시청 목록과 장르 유사도 높은 웹툰 중 평점 높은 10개

In [10]:
score = genre_df.to_numpy()

def genre_model(title_list):
    local_score = np.append(score, genres(title_list), axis=0)
    cosine_similar = cosine_similarity(local_score, local_score)
    cosine_similar_data = pd.DataFrame(cosine_similar)

    genre_user = cosine_similar_data.tail(1).T.sort_values(by=cosine_similar_data.columns[-1], ascending=False)
    max_score = genre_user.values[1][0]
    
    max_index = list(genre_user[(genre_user.values) == max_score].index)

    if df.shape[0] in max_index:
        max_index.remove(df.shape[0])

    sorted_df = df.loc[max_index].sort_values(by='score', ascending=False)

    if len(max_index) < 10:
        return sorted_df
    
    else:
        return sorted_df[:10]

In [11]:
genre_model(['바니와 오빠들', '신락도', '신의 탑', '오무라이스 잼잼'])

,title,score,드라마,로맨스,무협,스릴러,액션,일상,코믹,판타지,학원
2922,남편을 내 편으로 만드는 방법,9.990000,0,1,0,0,0,0,0,1,0
2226,창백한 말,9.990000,0,1,0,0,0,0,0,1,0
1928,샬롯에게는 다섯 명의 제자가 있다,9.990000,0,1,0,0,0,0,0,1,0
1973,우리 청춘의 사랑은,9.986667,0,0,0,0,0,1,1,0,0
2769,남아돌아,9.986667,0,0,0,0,0,1,1,0,0
1967,퀴퀴한 일기,9.986667,0,0,0,0,0,1,1,0,0
2002,반지하셋방,9.983333,0,0,0,0,0,1,1,0,0
2851,다정한 그대를 지키는 방법,9.983333,0,1,0,0,0,0,0,1,0
1968,열무와 알타리,9.983333,0,0,0,0,0,1,1,0,0
1852,오무라이스 잼잼,9.983333,0,0,0,0,0,1,1,0,0
